In [24]:
%%time
import numpy as np
from random import randint
import math
import pandas as pd
import pickle, os, math
from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers import Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

Wall time: 0 ns


In [36]:
%%time
data_dir = os.path.join(os.curdir, 'Data', 'Score', '400000_68per', 'data_basic.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics = ot['data']
target_pics = ot['target']
print(data_pics.shape)

X_train, X_test, y_train, y_test = train_test_split(
    data_pics, target_pics, test_size=0.1, random_state=randint(100, 10000))
print(X_train.shape)
X_train_train, X_vali, y_train_train, y_vali = train_test_split(
    X_train, y_train, test_size=0.3, random_state=randint(100, 10000))
print('class 0 has ' + str(len(y_train.index[y_train[0] == 0].tolist())) + ' points')
print('class 1 has ' + str(len(y_train.index[y_train[0] == 1].tolist())) + ' points')

index_0 = y_train.index[y_train[0] == 0].tolist()
index_1 = y_train.index[y_train[0] != 0].tolist()
index_0_comparable_to_1 = np.random.choice(index_0, math.floor(len(index_1) * 1))
samples = np.concatenate([index_1, index_0_comparable_to_1])
print(str(len(index_1)) + ' + ' + str(len(index_0_comparable_to_1)) + ' = ' + str(len(samples)))


small_data = data_pics.iloc[samples, :]
small_target = target_pics.iloc[samples, :]

X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    small_data, small_target, test_size=0.3, random_state=randint(100, 10000))
X_train_train_small, X_vali_small, y_train_train_small, y_vali_small = train_test_split(
    X_train_small, y_train_small, test_size=0.3, random_state=randint(100, 10000))

y_train_small_m = np.ravel(y_train_small)
y_test_small_m = np.ravel(y_test_small)
y_train_train_small_m = np.ravel(y_train_train_small)
y_vali_small_m = np.ravel(y_vali_small)
y_train_m = np.ravel(y_train)
y_test_m = np.ravel(y_test)
y_train_train_m = np.ravel(y_train_train)
y_vali_m = np.ravel(y_vali)

(90029, 221)
(81026, 221)
class 0 has 65183 points
class 1 has 15843 points
15843 + 15843 = 31686
Wall time: 871 ms


In [39]:
mlpc_st = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32, 16),
                        alpha=0.15, max_iter=1000, batch_size=5000,
                        verbose=True, learning_rate_init=0.04, tol=1e-5,
                        learning_rate='adaptive')

mlpc_st.fit(X_train_small, y_train_small_m)
mlpc_st.score(X_test_small, y_test_small_m)

Iteration 1, loss = 3.28341634
Iteration 2, loss = 0.72230461
Iteration 3, loss = 0.73042272
Iteration 4, loss = 0.70975215
Iteration 5, loss = 0.70465709
Iteration 6, loss = 0.73560345
Iteration 7, loss = 0.70587998
Iteration 8, loss = 0.69941877
Iteration 9, loss = 0.69033928
Iteration 10, loss = 0.68256500
Iteration 11, loss = 0.67955632
Iteration 12, loss = 0.67750325
Iteration 13, loss = 0.67529472
Iteration 14, loss = 0.67317561
Iteration 15, loss = 0.67118424
Iteration 16, loss = 0.67102048
Iteration 17, loss = 0.66935258
Iteration 18, loss = 0.66851202
Iteration 19, loss = 0.66693017
Iteration 20, loss = 0.67192036
Iteration 21, loss = 0.66257172
Iteration 22, loss = 0.66153741
Iteration 23, loss = 0.66050878
Iteration 24, loss = 0.65825059
Iteration 25, loss = 0.65835582
Iteration 26, loss = 0.66097896
Iteration 27, loss = 0.65768561
Iteration 28, loss = 0.65655742
Iteration 29, loss = 0.65419731
Iteration 30, loss = 0.65389481
Iteration 31, loss = 0.65144654
Iteration 32, los

0.6471702082895013

In [3]:
from sklearn.svm import LinearSVC

In [7]:
lsvc = LinearSVC(C=10.0, verbose=True, max_iter=20000)
lsvc.fit(X_train, y_train_m)
lsvc.score(X_test_small, y_test_small_m)

[LibLinear]

c:\users\hlu82\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.495332004615546

In [8]:
from sklearn.linear_model import SGDClassifier

In [26]:
sgdc = SGDClassifier(verbose=True, max_iter=20000, tol=1e-4, penalty='l2', learning_rate='optimal')
sgdc.fit(X_train_small, y_train_small_m)
sgdc.score(X_test_small, y_test_small_m)

-- Epoch 1
Norm: 218.97, NNZs: 204, Bias: -154.643969, T: 22092, Avg. loss: 259.634918
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 155.08, NNZs: 206, Bias: -67.204312, T: 44184, Avg. loss: 53.137236
Total training time: 0.04 seconds.
-- Epoch 3
Norm: 117.47, NNZs: 206, Bias: -46.664996, T: 66276, Avg. loss: 31.033312
Total training time: 0.06 seconds.
-- Epoch 4
Norm: 92.95, NNZs: 206, Bias: -34.049737, T: 88368, Avg. loss: 22.491312
Total training time: 0.08 seconds.
-- Epoch 5
Norm: 78.65, NNZs: 206, Bias: -28.885993, T: 110460, Avg. loss: 17.516258
Total training time: 0.10 seconds.
-- Epoch 6
Norm: 69.42, NNZs: 206, Bias: -22.851776, T: 132552, Avg. loss: 14.516721
Total training time: 0.12 seconds.
-- Epoch 7
Norm: 60.99, NNZs: 206, Bias: -18.403809, T: 154644, Avg. loss: 12.449717
Total training time: 0.14 seconds.
-- Epoch 8
Norm: 54.96, NNZs: 206, Bias: -18.059911, T: 176736, Avg. loss: 10.685782
Total training time: 0.16 seconds.
-- Epoch 9
Norm: 49.14, NNZs: 206, Bias

0.4977820025348543